In [97]:
%matplotlib inline

Generation of Questions with a seq2seq network 
*************************************************************




::

    [KEY: > input, = target, < output]

   

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Requirements**



In [98]:
from __future__ import unicode_literals, print_function, division
from io import open
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading data files
===================

The data for this project is a set of many thousands of sentence pairs, broken down into two main groups i.e agreement and no agreement and are present in the dataset folder of the project



Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

.. figure:: /_static/img/seq-seq-images/word-encoding.png
   :alt:





We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [99]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

we will make evertything lower case adn trim most punctuation.




In [100]:


# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = (s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs




In [101]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('./../data/agreement_data/train_data.txt' ,).\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

The maximum length of each sentence has been set to a 100 words.



In [102]:
MAX_LENGTH = 100



def filterPairs(pairs):
    return [pair for pair in pairs]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [103]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('out', 'inp', True)
print(random.choice(pairs))

Reading lines...
Read 435500 sentence pairs
Trimmed to 435500 sentence pairs
Counting words...
Counted words:
inp 54
out 52
['your cat does entertain your dog who doesnt smile quest', 'does your cat entertain your dog who doesnt smile']


The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [104]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





In [105]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)

Attention Decoder
^^^^^^^^^^^^^^^^^

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [106]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)



Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [107]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)9
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [108]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [109]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [110]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================



In [ ]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])


Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
param_groups 	 [{'momentum': 0.9, 'lr': 0.001, 'weight_decay': 0, 'nesterov': False, 'params': [140186999498144, 140186999497928, 140186999497568, 140186999498072, 140186999498216, 140186999498288, 140186999498432, 140186999498576, 140186999498360, 140186999498504], 'dampening': 0}]
state 	 {}


In [ ]:
model_exists = False
if model_exists :
    model = torch.load("no_agreement.pt")
    model.eval()

In [ ]:
hidden_size = 256
counter=1
while(counter < 11 ):
    name = "agreement_"+str(counter)+".pt"
    print(name)
    
    encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
    trainIters(encoder1, attn_decoder1, 75000, print_every=50)
    
    torch.save(model,name)   
    counter+=1

agreement_1.pt
0m 9s (- 225m 33s) (50 0%) 2.9303
0m 14s (- 179m 24s) (100 0%) 2.6591
0m 17s (- 148m 59s) (150 0%) 2.6203
0m 19s (- 120m 59s) (200 0%) 2.8250
0m 20s (- 103m 36s) (250 0%) 2.8578
0m 22s (- 92m 56s) (300 0%) 2.9198
0m 23s (- 84m 39s) (350 0%) 2.6377
0m 25s (- 78m 46s) (400 0%) 2.5528
0m 27s (- 74m 59s) (450 0%) 2.4674
0m 28s (- 71m 57s) (500 0%) 2.3478
0m 30s (- 69m 40s) (550 0%) 2.6012
0m 32s (- 67m 49s) (600 0%) 2.3876
0m 34s (- 65m 53s) (650 0%) 2.3598
0m 36s (- 64m 32s) (700 0%) 2.5275
0m 38s (- 62m 53s) (750 1%) 2.5943
0m 40s (- 61m 56s) (800 1%) 2.4275
0m 41s (- 60m 33s) (850 1%) 2.5808
0m 43s (- 59m 26s) (900 1%) 2.5360
0m 45s (- 58m 39s) (950 1%) 2.5525
0m 46s (- 57m 49s) (1000 1%) 2.4623
0m 48s (- 56m 36s) (1050 1%) 2.3862
0m 49s (- 55m 31s) (1100 1%) 2.4393
0m 50s (- 54m 29s) (1150 1%) 2.4901
0m 52s (- 53m 37s) (1200 1%) 2.2845
0m 53s (- 52m 47s) (1250 1%) 2.4459
0m 55s (- 52m 1s) (1300 1%) 2.4638
0m 56s (- 51m 22s) (1350 1%) 2.5648
0m 57s (- 50m 39s) (1400 1%) 2

5m 50s (- 32m 37s) (11400 15%) 0.2820
5m 52s (- 32m 35s) (11450 15%) 0.2424
5m 53s (- 32m 34s) (11500 15%) 0.4451
5m 55s (- 32m 32s) (11550 15%) 0.2387
5m 56s (- 32m 30s) (11600 15%) 0.2176
5m 58s (- 32m 29s) (11650 15%) 0.4054
6m 0s (- 32m 28s) (11700 15%) 0.2633
6m 1s (- 32m 26s) (11750 15%) 0.3084
6m 3s (- 32m 25s) (11800 15%) 0.2574
6m 4s (- 32m 23s) (11850 15%) 0.2488
6m 6s (- 32m 21s) (11900 15%) 0.2585
6m 7s (- 32m 19s) (11950 15%) 0.2751
6m 9s (- 32m 17s) (12000 16%) 0.2772
6m 10s (- 32m 15s) (12050 16%) 0.2250
6m 12s (- 32m 14s) (12100 16%) 0.3372
6m 13s (- 32m 12s) (12150 16%) 0.2622
6m 15s (- 32m 11s) (12200 16%) 0.2464
6m 16s (- 32m 9s) (12250 16%) 0.2764
6m 18s (- 32m 7s) (12300 16%) 0.2851
6m 19s (- 32m 5s) (12350 16%) 0.3599
6m 21s (- 32m 3s) (12400 16%) 0.3019
6m 22s (- 32m 2s) (12450 16%) 0.2760
6m 24s (- 32m 0s) (12500 16%) 0.2548
6m 25s (- 31m 58s) (12550 16%) 0.2933
6m 26s (- 31m 56s) (12600 16%) 0.2862
6m 28s (- 31m 54s) (12650 16%) 0.2528
6m 29s (- 31m 52s) (12700

11m 13s (- 26m 37s) (22250 29%) 0.1386
11m 15s (- 26m 36s) (22300 29%) 0.1177
11m 16s (- 26m 34s) (22350 29%) 0.0979
11m 18s (- 26m 33s) (22400 29%) 0.0941
11m 19s (- 26m 31s) (22450 29%) 0.0980
11m 21s (- 26m 30s) (22500 30%) 0.0958
11m 23s (- 26m 28s) (22550 30%) 0.1111
11m 24s (- 26m 27s) (22600 30%) 0.1046
11m 26s (- 26m 26s) (22650 30%) 0.1227
11m 28s (- 26m 25s) (22700 30%) 0.1058
11m 30s (- 26m 24s) (22750 30%) 0.2601
11m 31s (- 26m 23s) (22800 30%) 0.0963
11m 33s (- 26m 22s) (22850 30%) 0.1070
11m 35s (- 26m 21s) (22900 30%) 0.1208
11m 37s (- 26m 21s) (22950 30%) 0.1045
11m 40s (- 26m 23s) (23000 30%) 0.1724
11m 43s (- 26m 24s) (23050 30%) 0.1201
11m 44s (- 26m 23s) (23100 30%) 0.0977
11m 46s (- 26m 23s) (23150 30%) 0.1148
11m 48s (- 26m 22s) (23200 30%) 0.0917
11m 50s (- 26m 21s) (23250 31%) 0.0978
11m 52s (- 26m 20s) (23300 31%) 0.1288
11m 53s (- 26m 19s) (23350 31%) 0.1252
11m 55s (- 26m 18s) (23400 31%) 0.1526
11m 57s (- 26m 16s) (23450 31%) 0.0882
11m 59s (- 26m 16s) (2350

-306m 45s (- -395m 8s) (32700 43%) 0.0410
-306m 51s (- -394m 20s) (32750 43%) 0.0422
-306m 56s (- -393m 31s) (32800 43%) 0.0593
-305m 1s (- -392m 41s) (32850 43%) 0.0459
-305m 6s (- -391m 51s) (32900 43%) 0.0636
-305m 11s (- -389m 1s) (32950 43%) 0.0433
-305m 17s (- -388m 10s) (33000 44%) 0.0410
-305m 22s (- -387m 20s) (33050 44%) 0.0521
-305m 27s (- -386m 29s) (33100 44%) 0.0468
-305m 32s (- -385m 37s) (33150 44%) 0.0567
-305m 37s (- -384m 46s) (33200 44%) 0.0402
-305m 42s (- -383m 55s) (33250 44%) 0.0363
-305m 47s (- -381m 3s) (33300 44%) 0.0468
-305m 52s (- -380m 11s) (33350 44%) 0.0612
-305m 57s (- -379m 19s) (33400 44%) 0.0640
-304m 2s (- -378m 26s) (33450 44%) 0.0458
-304m 8s (- -377m 34s) (33500 44%) 0.0409
-304m 13s (- -376m 41s) (33550 44%) 0.0472
-304m 18s (- -375m 48s) (33600 44%) 0.0454
-304m 24s (- -374m 55s) (33650 44%) 0.0410
-304m 29s (- -372m 3s) (33700 44%) 0.0664
-304m 35s (- -371m 9s) (33750 45%) 0.0649
-304m 40s (- -370m 15s) (33800 45%) 0.0519
-304m 45s (- -369m 2

-289m 15s (- -223m 23s) (42350 56%) 0.0383
-289m 21s (- -222m 3s) (42400 56%) 0.0185
-289m 25s (- -222m 43s) (42450 56%) 0.0327
-289m 30s (- -221m 23s) (42500 56%) 0.0397
-289m 35s (- -220m 3s) (42550 56%) 0.0591
-289m 40s (- -220m 42s) (42600 56%) 0.0301
-289m 45s (- -219m 21s) (42650 56%) 0.0303
-289m 50s (- -218m 1s) (42700 56%) 0.0474
-289m 55s (- -218m 40s) (42750 56%) 0.0317
-288m 0s (- -217m 20s) (42800 57%) 0.0402
-288m 5s (- -217m 58s) (42850 57%) 0.0385
-288m 10s (- -216m 37s) (42900 57%) 0.0230
-288m 15s (- -215m 17s) (42950 57%) 0.0317
-288m 20s (- -215m 55s) (43000 57%) 0.0363
-288m 25s (- -214m 34s) (43050 57%) 0.0336
-288m 30s (- -213m 13s) (43100 57%) 0.0363
-288m 35s (- -213m 51s) (43150 57%) 0.0269
-288m 40s (- -212m 29s) (43200 57%) 0.0282
-288m 45s (- -211m 7s) (43250 57%) 0.0303
-288m 50s (- -211m 45s) (43300 57%) 0.0383
-288m 55s (- -210m 24s) (43350 57%) 0.0525
-288m 59s (- -209m 1s) (43400 57%) 0.0421
-287m 4s (- -209m 39s) (43450 57%) 0.0333
-287m 9s (- -208m 1

...

5m 21s (- 118m 18s) (3250 4%) 1.8093
5m 26s (- 118m 9s) (3300 4%) 1.7750
5m 31s (- 118m 2s) (3350 4%) 1.7612
5m 35s (- 117m 52s) (3400 4%) 1.7286
5m 40s (- 117m 44s) (3450 4%) 1.6981
5m 45s (- 117m 35s) (3500 4%) 1.6572
5m 50s (- 117m 29s) (3550 4%) 1.4629
5m 55s (- 117m 25s) (3600 4%) 1.6623
5m 59s (- 117m 15s) (3650 4%) 1.6026
6m 4s (- 117m 9s) (3700 4%) 1.6680
6m 9s (- 117m 1s) (3750 5%) 1.6210
6m 14s (- 116m 53s) (3800 5%) 1.4430
6m 19s (- 116m 51s) (3850 5%) 1.6156
6m 24s (- 116m 45s) (3900 5%) 1.5081
6m 29s (- 116m 37s) (3950 5%) 1.4448
6m 33s (- 116m 32s) (4000 5%) 1.4696
6m 38s (- 116m 25s) (4050 5%) 1.6167
6m 43s (- 116m 17s) (4100 5%) 1.3776
6m 48s (- 116m 14s) (4150 5%) 1.2986
6m 53s (- 116m 11s) (4200 5%) 1.4135
6m 58s (- 116m 7s) (4250 5%) 1.3348
7m 3s (- 116m 0s) (4300 5%) 1.3127
7m 8s (- 115m 51s) (4350 5%) 1.3891
7m 12s (- 115m 44s) (4400 5%) 1.3623
7m 17s (- 115m 38s) (4450 5%) 1.3272
7m 22s (- 115m 32s) (4500 6%) 1.3618
7m 27s (- 115m 30s) (4550 6%) 1.2127
7m 32s (- 1

23m 19s (- 102m 4s) (13950 18%) 0.2331
23m 24s (- 101m 58s) (14000 18%) 0.2495
23m 29s (- 101m 53s) (14050 18%) 0.2160
23m 34s (- 101m 49s) (14100 18%) 0.2438
23m 39s (- 101m 44s) (14150 18%) 0.2922
23m 44s (- 101m 40s) (14200 18%) 0.2345
23m 49s (- 101m 36s) (14250 19%) 0.2605
23m 55s (- 101m 32s) (14300 19%) 0.2340
24m 0s (- 101m 27s) (14350 19%) 0.2741
24m 5s (- 101m 21s) (14400 19%) 0.2624
24m 10s (- 101m 17s) (14450 19%) 0.2162
24m 15s (- 101m 12s) (14500 19%) 0.2386
24m 20s (- 101m 6s) (14550 19%) 0.2184
24m 25s (- 101m 2s) (14600 19%) 0.2452
24m 30s (- 100m 57s) (14650 19%) 0.3011
24m 35s (- 100m 53s) (14700 19%) 0.2402
24m 40s (- 100m 48s) (14750 19%) 0.2039
24m 45s (- 100m 43s) (14800 19%) 0.2203
24m 50s (- 100m 38s) (14850 19%) 0.2147
24m 56s (- 100m 34s) (14900 19%) 0.2336
25m 1s (- 100m 30s) (14950 19%) 0.2341
25m 6s (- 100m 26s) (15000 20%) 0.2369
25m 11s (- 100m 21s) (15050 20%) 0.2579
25m 16s (- 100m 17s) (15100 20%) 0.2114
25m 21s (- 100m 11s) (15150 20%) 0.2168
25m 26s

41m 24s (- 85m 5s) (24550 32%) 0.0962
41m 29s (- 85m 0s) (24600 32%) 0.0683
41m 34s (- 84m 54s) (24650 32%) 0.1461
41m 39s (- 84m 49s) (24700 32%) 0.0699
41m 44s (- 84m 45s) (24750 33%) 0.1268
41m 49s (- 84m 39s) (24800 33%) 0.0497
41m 54s (- 84m 34s) (24850 33%) 0.0794
41m 59s (- 84m 29s) (24900 33%) 0.1232
42m 4s (- 84m 24s) (24950 33%) 0.0699
42m 9s (- 84m 19s) (25000 33%) 0.1016
42m 15s (- 84m 15s) (25050 33%) 0.0984
42m 20s (- 84m 10s) (25100 33%) 0.0519
42m 25s (- 84m 5s) (25150 33%) 0.0851
42m 30s (- 83m 59s) (25200 33%) 0.1145
42m 35s (- 83m 54s) (25250 33%) 0.0980
42m 40s (- 83m 49s) (25300 33%) 0.1001
42m 45s (- 83m 44s) (25350 33%) 0.1028
42m 50s (- 83m 40s) (25400 33%) 0.1378
42m 55s (- 83m 35s) (25450 33%) 0.1042
43m 0s (- 83m 29s) (25500 34%) 0.0914
43m 6s (- 83m 25s) (25550 34%) 0.0853
43m 11s (- 83m 20s) (25600 34%) 0.1159
43m 16s (- 83m 15s) (25650 34%) 0.1061
43m 21s (- 83m 9s) (25700 34%) 0.1380
43m 26s (- 83m 5s) (25750 34%) 0.1258
43m 32s (- 83m 1s) (25800 34%) 0.0

59m 26s (- 67m 22s) (35150 46%) 0.0362
59m 31s (- 67m 17s) (35200 46%) 0.0536
59m 36s (- 67m 12s) (35250 47%) 0.0402
59m 41s (- 67m 7s) (35300 47%) 0.0326
59m 46s (- 67m 2s) (35350 47%) 0.0289
59m 51s (- 66m 57s) (35400 47%) 0.0591
59m 56s (- 66m 52s) (35450 47%) 0.0320
60m 2s (- 66m 47s) (35500 47%) 0.0304
60m 7s (- 66m 42s) (35550 47%) 0.0499
60m 12s (- 66m 38s) (35600 47%) 0.0458
60m 17s (- 66m 33s) (35650 47%) 0.0316
60m 22s (- 66m 28s) (35700 47%) 0.0534
60m 28s (- 66m 23s) (35750 47%) 0.0427
60m 33s (- 66m 18s) (35800 47%) 0.0423
60m 38s (- 66m 13s) (35850 47%) 0.0563
60m 43s (- 66m 8s) (35900 47%) 0.0458
60m 48s (- 66m 3s) (35950 47%) 0.0469
60m 53s (- 65m 57s) (36000 48%) 0.0441
60m 58s (- 65m 52s) (36050 48%) 0.0330
61m 3s (- 65m 47s) (36100 48%) 0.0538
61m 8s (- 65m 42s) (36150 48%) 0.0467
61m 13s (- 65m 37s) (36200 48%) 0.0399
61m 18s (- 65m 32s) (36250 48%) 0.0439
61m 23s (- 65m 27s) (36300 48%) 0.0398
61m 29s (- 65m 22s) (36350 48%) 0.0279
61m 34s (- 65m 17s) (36400 48%) 0

77m 34s (- 49m 36s) (45750 61%) 0.0277
77m 39s (- 49m 30s) (45800 61%) 0.0610
77m 45s (- 49m 25s) (45850 61%) 0.0589
77m 50s (- 49m 20s) (45900 61%) 0.0302
77m 55s (- 49m 15s) (45950 61%) 0.0580
78m 0s (- 49m 10s) (46000 61%) 0.0918
78m 5s (- 49m 5s) (46050 61%) 0.0361
78m 11s (- 49m 0s) (46100 61%) 0.0843
78m 16s (- 48m 55s) (46150 61%) 0.0312
78m 21s (- 48m 50s) (46200 61%) 0.0559
78m 26s (- 48m 45s) (46250 61%) 0.0281
78m 31s (- 48m 40s) (46300 61%) 0.0355
78m 36s (- 48m 35s) (46350 61%) 0.0458
78m 41s (- 48m 30s) (46400 61%) 0.0404
78m 46s (- 48m 25s) (46450 61%) 0.0615
78m 52s (- 48m 20s) (46500 62%) 0.0428
78m 57s (- 48m 15s) (46550 62%) 0.0297
79m 2s (- 48m 10s) (46600 62%) 0.0314
79m 7s (- 48m 5s) (46650 62%) 0.0305
79m 12s (- 47m 59s) (46700 62%) 0.0267
79m 17s (- 47m 54s) (46750 62%) 0.0354
79m 23s (- 47m 50s) (46800 62%) 0.0827
79m 28s (- 47m 44s) (46850 62%) 0.0163
79m 33s (- 47m 39s) (46900 62%) 0.0151
79m 38s (- 47m 34s) (46950 62%) 0.0159
79m 43s (- 47m 29s) (47000 62%) 

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "her rabbit who would irritate our rabbit could smile ident")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    fig.savefig(input_sentence+'.png')


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("your cat yak smile ident")

